In [1]:
# Importing sqlalchemy
import sqlalchemy as sql

<h1>Databases</h1>

Databases are powerful tools to store and organize data. They take advantage of their design (schema) to provide efficient ways to retrieve the stored data and determine relationships between different tables.<br>

In this sense, one of the most popular choices to create databases is the Structured Query Language (SQL). In Python, SQL can be rather easily used with the [SQLAlchemy](https://www.sqlalchemy.org/) library.

In this tutorial we are going to cover the basic operations that you will find and use being those the creation of the database, querying, updating and deleting.

<h1>Designing a database</h1>

The code block below showcase how to create a database and define the items that are being used on it ([Example](https://www.pythoncentral.io/introductory-tutorial-python-sqlalchemy/)). Notice that each table is contained within a [class](https://www.w3schools.com/python/python_classes.asp).


In [7]:
# Some more imports.
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

In [3]:
Base = declarative_base()

class Pokemon(Base):
    __tablename__ = 'pokemon'
    # Here we define columns for the table person
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)

class Attributes(Base):
    __tablename__ = 'attributes'
    # Here we define columns for the table address.
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    type1 = Column(String(50))
    type2 = Column(String(50))
    HP = Column(Integer, nullable=False)
    Attack = Column(Integer, nullable=False)
    Legendary = Column(Boolean, nullable=False)
    pokemon_id = Column(Integer, ForeignKey('pokemon.id'))
    Pokemon = relationship(Pokemon)

# Create an engine that stores data in the local directory's
# sqlalchemy_example.db file.
engine = create_engine('sqlite:///pokedex.db')

# Create all tables in the engine. This is equivalent to "Create Table"
# statements in raw SQL.
Base.metadata.create_all(engine)

In [10]:
# print working directory




<module 'pwd' (built-in)>

Discussion: are the assigned datatypes of the table above appropriated to the problem?

Exercise: the pokemon dataframe had more columns than what we can see in the attributes class. Add the columns for the attributes.

In [4]:
# Having a peek at the object we created.
Pokemon

__main__.Pokemon

<h1>Starting a database session</h1>

First thing we need to do is to start a database [session](https://docs.sqlalchemy.org/en/14/orm/session_api.html). After that we are able to start working with the database. Just be mindful that this connection (ideally) needs to be closed later.


If you prefer, SQLAlchemy can also work with [PostgreSQL](https://docs.sqlalchemy.org/en/14/dialects/postgresql.html), [MySQL](https://docs.sqlalchemy.org/en/14/dialects/mysql.html) and other formats of databases. In this tutorial we will use [SQLite](https://docs.sqlalchemy.org/en/14/dialects/sqlite.html).

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

#from sqlalchemy_declarative import Attributes, Base, Pokemon

# Notice we're using SQLite here.
# This line will create the database if it isn't there already.
engine = create_engine('sqlite:///pokedex.db')
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()

<h1>Inserting data on the database</h1>
Here is how we take data into the database. This proccess can be simple refered to as [add](https://docs.sqlalchemy.org/en/13/orm/session_basics.html#adding-new-or-existing-items). Again, any additions need to be commited before they actually become part of the database.

In [ ]:
# Insert a Person in the person table
new_pokemon = Pokemon(name='Bulbasaur')
session.add(new_pokemon)
session.commit()

In [ ]:
# Insert an Address in the address table
new_type = Attributes(type1='grass', Pokemon=new_pokemon)
session.add(new_type)
session.commit()

Exercise: Why has the cell block above crashed? (hint: read the error message).

In [ ]:
# Fixing the crash.
session.rollback()

In [ ]:
# Insert an Address in the address table
new_type = Attributes(type1='grass', 
                      type2=None, 
                      HP=45, 
                      Attack=49, 
                      Legendary=False, 
                      Pokemon=new_pokemon)
session.add(new_type)
session.commit()

Exercise: add 5 more pokemon to the database.

<h1>Querying</h1>

This is generally where database users will spend most of their time. Efficiently [Querying](https://docs.sqlalchemy.org/en/13/orm/session_basics.html#querying) the database is remarkably important to save processing time and, of course, retrieve the right data.

In [ ]:
# Example querying the pokemon database.
s = sql.select([Pokemon])
rp = session.execute(s)
results = rp.fetchall()
results

In [ ]:
# Example querying the Attributes database.
s = sql.select([Attributes])
rp = session.execute(s)
results = rp.fetchall()
results

The queries can also be filtered using a where statement.

In [ ]:
# Example with a where statement.
s = sql.select([Attributes]).where(Attributes.pokemon_id == 1)
rp = session.execute(s)
results = rp.fetchall()
results

Raw SQL is also understood by sqlalchemy as in the examples below,

In [ ]:
s = "SELECT * FROM pokemon"
rp = session.execute(s)
results = rp.fetchall()
results

In [ ]:
s = "SELECT * FROM pokemon WHERE ID = 1"
rp = session.execute(s)
results = rp.fetchall()
results

Exercise: Your turn to make some queries. Show two examples of filtered queries.

<h1>Updating</h1>
Sometimes you might want to change data that is stored in the database.

In [ ]:
# Notice anything different about the query?
s = session.query(Attributes)
s = s.filter(Attributes.pokemon_id == 1)

# Update example.
s.update({Attributes.HP: 500})
session.commit()

Exercise: Yout turn to update a value.

<h1>Deleting</h1>
Finally let's look at how to delete entries in the database.

In [ ]:
# Delete example
s = sql.delete(Attributes).where(Attributes.pokemon_id == 1)
rp = session.execute(s)

Discussion: What are the dangers of deleting items?

Exercise: Delete a row.

In [ ]:
# Closing session.
session.close()